# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, Dataset, Model, Environment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


from sklearn.model_selection import train_test_split
import pandas as pd
import os
import json
import joblib
import sklearn

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

### Kaggle - Housing Prices Competition for Kaggle Learn Users
We will be using the "Housing Prices Competition for Kaggle Learn Users" training and test datasets for this capstone project.

This is a regression competition in which competitors try to predict the price of the houses in the test dataset using the training dataset.

The original dataset was first published by Dean De Cock in his paper [Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project](https://www.researchgate.net/publication/267976209_Ames_Iowa_Alternative_to_the_Boston_Housing_Data_as_an_End_of_Semester_Regression_Project) at Journal of Statistics Education (November 2011).

For competition purposes, approximately all of the data has been divided into two parts: "training dataset" and "test dataset" We will be using the training dataset for training and the test dataset for submission to the competition.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A4VZYSPRN to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-137888
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-137888


In [3]:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py#provisioning-configuration-vm-size-----vm-priority--dedicated---min-nodes-0--max-nodes-none--idle-seconds-before-scaledown-none--admin-username-none--admin-user-password-none--admin-user-ssh-key-none--vnet-resourcegroup-name-none--vnet-name-none--subnet-name-none--tags-none--description-none--remote-login-port-public-access--notspecified--

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb


# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.pipeline import Pipeline

#global X_train, X_test, y_train, y_test

def clean_data(data, test):

    # Convert dataset to pandas dataframe
    X = data.to_pandas_dataframe()
    X_test = test.to_pandas_dataframe()
    # Set Id to index
    X.set_index('Id',inplace=True)
    X_test.set_index('Id',inplace=True)
    # Remove rows with missing target, separate target from predictors
    X.dropna(axis=0, subset=['SalePrice'], inplace=True)
    y = X.SalePrice 
    # Remove target and 'Utilities' 
    X.drop(['SalePrice', 'Utilities'], axis=1, inplace=True)
    X_test.drop(['Utilities'], axis=1, inplace=True)
    # Split the data
    X_train, X_valid, y_train, y_valid = train_test_split(X,y)
    # Select object columns
    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
    # Select numeric columns
    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64','float64']]

    # Imputation lists
    # imputation to null values of these numerical columns need to be 'constant'
    constant_num_cols = ['GarageYrBlt', 'MasVnrArea']
    # imputation to null values of these numerical columns need to be 'mean'
    mean_num_cols = list(set(numerical_cols).difference(set(constant_num_cols)))
    # imputation to null values of these categorical columns need to be 'constant'
    constant_categorical_cols = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']
    # imputation to null values of these categorical columns need to be 'most_frequent'
    mf_categorical_cols = list(set(categorical_cols).difference(set(constant_categorical_cols)))

    my_cols = constant_num_cols + mean_num_cols + constant_categorical_cols + mf_categorical_cols

    # Define transformers
    # Preprocessing for numerical data - mean
    numerical_transformer_m = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('scaler', StandardScaler())])
    # Preprocessing for numerical data - constant
    numerical_transformer_c = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0)),('scaler', StandardScaler())])

    # Preprocessing for categorical data for most frequent
    categorical_transformer_mf = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse = False))])
    # Preprocessing for categorical data for constant
    categorical_transformer_c = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='NA')), ('onehot', OneHotEncoder(handle_unknown = 'ignore', sparse = False))])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(transformers=[
        ('num_mean', numerical_transformer_m, mean_num_cols),
        ('num_constant', numerical_transformer_c, constant_num_cols),
        ('cat_mf', categorical_transformer_mf, mf_categorical_cols),
        ('cat_c', categorical_transformer_c, constant_categorical_cols)])

    # Transform data
    X_train = preprocessor.fit_transform(X_train)
    X_valid = preprocessor.transform(X_valid)
    X_test = preprocessor.transform(X_test)
    
    
    # Concat datasets
    # https://stackoverflow.com/questions/41989950/numpy-array-concatenate-valueerror-all-the-input-arrays-must-have-same-number
    train_data = np.concatenate([X_train, y_train[:,None]], axis=1)
    valid_data = np.concatenate([X_valid, y_valid[:,None]], axis=1)
    
    
    # Return data
    return train_data, valid_data, X_test

In [7]:
# Get the dataset
ds_train = Dataset.get_by_name(ws, name='Housing Prices Dataset')
ds_test = Dataset.get_by_name(ws, name='Housing Prices Test Dataset')

# Use the clean_data function to clean your data.
train_data, valid_data, test_data = clean_data(ds_train, ds_test)
print (train_data.shape)
print (valid_data.shape)
print(test_data.shape)

(1095, 402)
(365, 402)
(1459, 401)


In [13]:
train_data[1,401]

160000.0

In [14]:
# automl_config requires TabularDataset as a result we need to
# create a dataset from pandas dataframe
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#create-a-filedataset
print(type(train_data))

# create data folder if not exist 
if "data" not in os.listdir():
    os.mkdir("./data")

# convert train dataframe
# https://stackoverflow.com/questions/11106536/adding-row-column-headers-to-numpy-arrays
number_of_columns=len(train_data[1,:])
names = [i for i in range(number_of_columns)]
test_ds_names = [i for i in range(number_of_columns-1)]
train_path = 'data/train_cleaned.csv'
cleaned_train_data = pd.DataFrame(train_data, columns=names)
cleaned_train_data.to_csv(train_path, index=False, header=True, sep=',')

# convert valid dataframe
valid_path = 'data/valid_cleaned.csv'
cleaned_valid_data = pd.DataFrame(valid_data, columns=names)
cleaned_valid_data.to_csv(valid_path, index=False, header=True, sep=',')

# convert test dataframe
test_path = 'data/test_cleaned.csv'
cleaned_test_data = pd.DataFrame(test_data, columns=test_ds_names)
cleaned_test_data.to_csv(test_path, index=False, header=True, sep=',')

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
train_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/train_cleaned.csv'))])

# create a dataset referencing the cloud location
valid_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/valid_cleaned.csv'))])

# create a dataset referencing the cloud location
test_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/test_cleaned.csv'))])

<class 'numpy.ndarray'>
Uploading an estimated of 7 files
Uploading data/data_description.txt
Uploaded data/data_description.txt, 1 files out of an estimated total of 7
Uploading data/sample_submission.csv
Uploaded data/sample_submission.csv, 2 files out of an estimated total of 7
Uploading data/test.csv
Uploaded data/test.csv, 3 files out of an estimated total of 7
Uploading data/test_cleaned.csv
Uploaded data/test_cleaned.csv, 4 files out of an estimated total of 7
Uploading data/train.csv
Uploaded data/train.csv, 5 files out of an estimated total of 7
Uploading data/train_cleaned.csv
Uploaded data/train_cleaned.csv, 6 files out of an estimated total of 7
Uploading data/valid_cleaned.csv
Uploaded data/valid_cleaned.csv, 7 files out of an estimated total of 7
Uploaded 7 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
print(valid_dataset.to_pandas_dataframe().head(1))

          0         1         2         3         4        5         6  \
0 -0.512255 -0.113566 -0.706048 -0.060408 -0.986004 -0.10477  0.770368   

          7         8        9  ...  392  393  394  395  396  397  398  399  \
0 -0.579391 -0.801803 -0.30062  ...  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   

   400       401  
0  0.0  118000.0  

[1 rows x 402 columns]


In [16]:
project_folder = './'
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "max_cores_per_iteration":-1,
    "max_concurrent_iterations":4, 
    "n_cross_validations":5,
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = ws.compute_targets['cpu-cluster'],
                             task = "regression",
                             primary_metric = 'normalized_root_mean_squared_error',
                             training_data=train_dataset,
                             validation_data=valid_dataset,
                             label_column_name="401",   
                             path = project_folder
                            )

In [17]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [18]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
assert(remote_run.get_status()=="Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

******************************************

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
# Retrieve and save your best automl model.

# https://github.com/MicrosoftLearning/DP100/blob/master/08B%20-%20Using%20Automated%20Machine%20Learning.ipynb
# Get the best run object
best_run, fitted_model = remote_run.get_output()
print("Summary:")
print(remote_run.summary())
print("********************\n")
print("Best run:")
print(best_run)
print("********************\n")
print("Estimator:")
print(fitted_model.steps[-1])
print("********************\n")
print("Model:")
print(fitted_model)
print("********************\n")
best_run_metrics = best_run.get_metrics()
print('MAE:', best_run_metrics['mean_absolute_error'])
print('RMSLE:', best_run_metrics['root_mean_squared_log_error'])

print("********************\n")

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Summary:
[['StackEnsemble', 1, 0.030418459417518372], ['VotingEnsemble', 1, 0.028668891624893843], ['ExtremeRandomTrees', 3, 0.04453343091533696], ['RandomForest', 7, 0.03558283728211917], ['GradientBoosting', 5, 0.030669436576262758], ['XGBoostRegressor', 2, 0.03245968344900179], ['DecisionTree', 16, 0.046399180289105425], ['LassoLars', 2, 0.03264838233928287], ['ElasticNet', 3, 0.03702390251961543], ['LightGBM', 1, 0.034077517371056916]]
********************

Best run:
Run(Experiment: automl_experiment,
Id: AutoML_8aa12d5a-a272-4123-9ca3-d6daaf8c4a0f_39,
Type: azureml.scriptrun,
Status: Completed)
********************

Estimator:
('prefittedsoftvotingregressor', PreFittedSoftVotingRegressor(estimators=[('28',
                                          Pipeline(memory=None,
                                                   steps=[('standardscalerwrapper',
                                                           <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper obje

In [21]:
#TODO: Save the best model
# https://knowledge.udacity.com/questions/357007
os.makedirs('outputs', exist_ok=True)
joblib.dump(fitted_model, 'automl_model.pkl')


['automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
# Register model
model = Model.register(workspace= ws,model_path='automl_model.pkl', model_name='best_automl_run')
# Check model
for model in Model.list(ws):
    print("Model Name: {}\n".format(model.name))
    print(model)
    print("********************\n")

Registering model best_automl_run
Model Name: best_automl_run

Model(workspace=Workspace.create(name='quick-starts-ws-137888', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-137888'), name=best_automl_run, id=best_automl_run:1, version=1, tags={}, properties={})
********************



In [23]:
env = Environment.get(workspace=ws, name="AzureML-AutoML")

In [24]:
print("packages", env.python.conda_dependencies.serialize_to_string())

packages channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.21.0.post1
  - azureml-pipeline-core==1.21.0
  - azureml-telemetry==1.21.0
  - azureml-defaults==1.21.0
  - azureml-interpret==1.21.0
  - azureml-automl-core==1.21.0
  - azureml-automl-runtime==1.21.0
  - azureml-train-automl-client==1.21.0
  - azureml-train-automl-runtime==1.21.0.post1
  - azureml-dataset-runtime==1.21.0
  - inference-schema
  - py-cpuinfo==5.0.0
  - boto3==1.15.18
  - botocore==1.18.18
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_7ade26eb614f97df8030bc480da59236



In [66]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- joblib
- numpy
- pip:
  - azureml-defaults
  - inference-schema[numpy-support]
  - azureml-train-automl
  - xgboost

Overwriting conda_dependencies.yml


In [25]:
from azureml.core import Environment

my_env = Environment.from_conda_specification(name = 'my-env', file_path = './my-env.yml')

In [26]:
with open('score.py') as f:
    print(f.read())

import joblib
import numpy as np
import os
import json

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


# The init() method is called once, when the web service starts up.
#
# Typically you would deserialize the model file, as shown here using joblib,
# and store it in a global variable so your run() method can access it later.
def init():
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_filename = 'automl_model.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)

    model = joblib.load(model_path)


# The run() method is called each time a request is made to the scoring API.
#
# Shown here are the optional input_schema and output_schema decorators
# from the inference-schema pip package. Using these decorators on your
# run() method parses and 

In [27]:
service_name = 'my-automl-service'
#my_model = Model(ws, 'best_automl_run', version=1)
my_model = Model(ws, 'best_automl_run')
inference_config = InferenceConfig(entry_script='score.py', environment=my_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[my_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [49]:
# Enable application insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [28]:
my_list= [0.3724756684108134,-0.11356630391730027,-0.7060478345151526,-0.06040800875041221,-0.6540846522394981,-0.10477002014272219,0.035279110958348174,-0.34837583243057135,-0.8018028001600404,0.5860246521733481,-1.0728719873198258,1.62293272927206,-0.21551091417475915,-0.776769403984661,-1.1813162093917609,-0.9439279700023309,-0.9705838675587369,1.8109138810402308,-0.11485318283789403,-1.044499739955759,-0.8041782736240765,-0.36546741585759634,0.0976573867257759,-1.182014706255381,-0.6849259227844027,-0.10346810050552385,0.36530285066800683,-0.8683613109578018,-0.23877727469036084,-0.39582858239823265,-0.8325132083251996,-1.0381841999958483,1.163263052393548,0.20313189092138595,-0.5712774912947869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0]
my_data={}
print(len(my_list))
for count in range(len(my_list)):
    my_data[str(count)]=my_list[count]
print(my_data)

401
{'0': 0.3724756684108134, '1': -0.11356630391730027, '2': -0.7060478345151526, '3': -0.06040800875041221, '4': -0.6540846522394981, '5': -0.10477002014272219, '6': 0.035279110958348174, '7': -0.34837583243057135, '8': -0.8018028001600404, '9': 0.5860246521733481, '10': -1.0728719873198258, '11': 1.62293272927206, '12': -0.21551091417475915, '13': -0.776769403984661, '14': -1.1813162093917609, '15': -0.9439279700023309, '16': -0.9705838675587369, '17': 1.8109138810402308, '18': -0.11485318283789403, '19': -1.044499739955759, '20': -0.8041782736240765, '21': -0.36546741585759634, '22': 0.0976573867257759, '23': -1.182014706255381, '24': -0.6849259227844027, '25': -0.10346810050552385, '26': 0.36530285066800683, '27': -0.8683613109578018, '28': -0.23877727469036084, '29': -0.39582858239823265, '30': -0.8325132083251996, '31': -1.0381841999958483, '32': 1.163263052393548, '33': 0.20313189092138595, '34': -0.5712774912947869, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 

In [29]:
data = {"data":
        [
          {'0': 0.3724756684108134, '1': -0.11356630391730027, '2': -0.7060478345151526, '3': -0.06040800875041221, '4': -0.6540846522394981, '5': -0.10477002014272219,
          '6': 0.035279110958348174, '7': -0.34837583243057135, '8': -0.8018028001600404, '9': 0.5860246521733481, '10': -1.0728719873198258, '11': 1.62293272927206,
          '12': -0.21551091417475915, '13': -0.776769403984661, '14': -1.1813162093917609, '15': -0.9439279700023309, '16': -0.9705838675587369, '17': 1.8109138810402308,
          '18': -0.11485318283789403, '19': -1.044499739955759, '20': -0.8041782736240765, '21': -0.36546741585759634, '22': 0.0976573867257759, '23': -1.182014706255381,
          '24': -0.6849259227844027, '25': -0.10346810050552385, '26': 0.36530285066800683, '27': -0.8683613109578018, '28': -0.23877727469036084, '29': -0.39582858239823265,
          '30': -0.8325132083251996, '31': -1.0381841999958483, '32': 1.163263052393548, '33': 0.20313189092138595, '34': -0.5712774912947869, '35': 0.0, '36': 0.0, '37': 0.0,
          '38': 0.0, '39': 0.0, '40': 0.0, '41': 0.0, '42': 0.0, '43': 0.0, '44': 0.0, '45': 0.0, '46': 0.0, '47': 1.0, '48': 0.0, '49': 0.0, '50': 0.0, '51': 0.0, '52': 0.0,
          '53': 0.0, '54': 0.0, '55': 0.0, '56': 0.0, '57': 0.0, '58': 0.0, '59': 0.0, '60': 0.0, '61': 1.0, '62': 0.0, '63': 0.0, '64': 0.0, '65': 0.0, '66': 0.0, '67': 0.0,
          '68': 0.0, '69': 0.0, '70': 0.0, '71': 0.0, '72': 1.0, '73': 0.0, '74': 1.0, '75': 0.0, '76': 0.0, '77': 0.0, '78': 0.0, '79': 0.0, '80': 0.0, '81': 0.0, '82': 0.0,
          '83': 0.0, '84': 0.0, '85': 0.0, '86': 0.0, '87': 0.0, '88': 0.0, '89': 0.0, '90': 0.0, '91': 0.0, '92': 1.0, '93': 0.0, '94': 0.0, '95': 0.0, '96': 1.0, '97': 0.0,
          '98': 1.0, '99': 0.0, '100': 0.0, '101': 0.0, '102': 0.0, '103': 0.0, '104': 0.0, '105': 0.0, '106': 0.0, '107': 0.0, '108': 0.0, '109': 0.0, '110': 0.0, '111': 1.0,
          '112': 0.0, '113': 0.0, '114': 0.0, '115': 0.0, '116': 0.0, '117': 1.0, '118': 0.0, '119': 0.0, '120': 0.0, '121': 1.0, '122': 0.0, '123': 0.0, '124': 1.0, '125': 0.0,
          '126': 0.0, '127': 0.0, '128': 0.0, '129': 0.0, '130': 0.0, '131': 0.0, '132': 0.0, '133': 1.0, '134': 1.0, '135': 0.0, '136': 0.0, '137': 0.0, '138': 0.0, '139': 0.0,
          '140': 0.0, '141': 0.0, '142': 1.0, '143': 0.0, '144': 0.0, '145': 0.0, '146': 1.0, '147': 0.0, '148': 1.0, '149': 0.0, '150': 0.0, '151': 0.0, '152': 0.0, '153': 0.0,
          '154': 0.0, '155': 0.0, '156': 0.0, '157': 0.0, '158': 0.0, '159': 1.0, '160': 0.0, '161': 0.0, '162': 1.0, '163': 0.0, '164': 0.0, '165': 0.0, '166': 0.0, '167': 1.0, '168': 0.0,
          '169': 0.0, '170': 0.0, '171': 0.0, '172': 1.0, '173': 0.0, '174': 0.0, '175': 0.0, '176': 0.0, '177': 0.0, '178': 0.0, '179': 1.0, '180': 0.0, '181': 0.0, '182': 0.0, '183': 0.0,
          '184': 1.0, '185': 0.0, '186': 0.0, '187': 0.0, '188': 0.0, '189': 0.0, '190': 0.0, '191': 0.0, '192': 0.0, '193': 0.0, '194': 0.0, '195': 0.0, '196': 0.0, '197': 0.0, '198': 0.0,
          '199': 0.0, '200': 0.0, '201': 0.0, '202': 0.0, '203': 0.0, '204': 0.0, '205': 0.0, '206': 0.0, '207': 0.0, '208': 0.0, '209': 0.0, '210': 0.0, '211': 0.0, '212': 0.0, '213': 0.0,
          '214': 0.0, '215': 0.0, '216': 0.0, '217': 0.0, '218': 0.0, '219': 0.0, '220': 0.0, '221': 0.0, '222': 0.0, '223': 0.0, '224': 0.0, '225': 0.0, '226': 0.0, '227': 0.0, '228': 0.0,
          '229': 0.0, '230': 0.0, '231': 0.0, '232': 0.0, '233': 0.0, '234': 0.0, '235': 0.0, '236': 0.0, '237': 0.0, '238': 0.0, '239': 0.0, '240': 0.0, '241': 0.0, '242': 0.0, '243': 0.0,
          '244': 0.0, '245': 0.0, '246': 0.0, '247': 0.0, '248': 0.0, '249': 0.0, '250': 0.0, '251': 0.0, '252': 0.0, '253': 0.0, '254': 0.0, '255': 0.0, '256': 0.0, '257': 0.0, '258': 0.0,
          '259': 0.0, '260': 0.0, '261': 0.0, '262': 0.0, '263': 0.0, '264': 0.0, '265': 0.0, '266': 0.0, '267': 0.0, '268': 0.0, '269': 0.0, '270': 0.0, '271': 1.0, '272': 0.0, '273': 0.0, 
          '274': 0.0, '275': 0.0, '276': 0.0, '277': 0.0, '278': 0.0, '279': 0.0, '280': 0.0, '281': 0.0, '282': 0.0, '283': 0.0, '284': 0.0, '285': 0.0, '286': 0.0, '287': 0.0, '288': 0.0,
          '289': 0.0, '290': 0.0, '291': 0.0, '292': 0.0, '293': 1.0, '294': 0.0, '295': 0.0, '296': 0.0, '297': 0.0, '298': 0.0, '299': 0.0, '300': 0.0, '301': 0.0, '302': 0.0, '303': 0.0, 
          '304': 0.0, '305': 0.0, '306': 0.0, '307': 0.0, '308': 0.0, '309': 1.0, '310': 0.0, '311': 0.0, '312': 0.0, '313': 0.0, '314': 1.0, '315': 0.0, '316': 0.0, '317': 0.0, '318': 0.0, 
          '319': 1.0, '320': 0.0, '321': 0.0, '322': 1.0, '323': 0.0, '324': 0.0, '325': 0.0, '326': 0.0, '327': 1.0, '328': 0.0, '329': 0.0, '330': 0.0, '331': 0.0, '332': 0.0, '333': 1.0, 
          '334': 0.0, '335': 0.0, '336': 0.0, '337': 0.0, '338': 1.0, '339': 0.0, '340': 0.0, '341': 0.0, '342': 0.0, '343': 1.0, '344': 0.0, '345': 0.0, '346': 0.0, '347': 0.0, '348': 0.0, 
          '349': 1.0, '350': 0.0, '351': 0.0, '352': 0.0, '353': 0.0, '354': 1.0, '355': 0.0, '356': 0.0, '357': 0.0, '358': 0.0, '359': 0.0, '360': 0.0, '361': 1.0, '362': 0.0, '363': 0.0, 
          '364': 0.0, '365': 1.0, '366': 0.0, '367': 0.0, '368': 0.0, '369': 0.0, '370': 0.0, '371': 0.0, '372': 0.0, '373': 0.0, '374': 1.0, '375': 0.0, '376': 0.0, '377': 0.0, '378': 0.0, 
          '379': 0.0, '380': 1.0, '381': 0.0, '382': 0.0, '383': 0.0, '384': 0.0, '385': 0.0, '386': 1.0, '387': 0.0, '388': 0.0, '389': 0.0, '390': 1.0, '391': 0.0, '392': 0.0, '393': 1.0, 
          '394': 0.0, '395': 0.0, '396': 0.0, '397': 1.0, '398': 0.0, '399': 0.0, '400': 0.0},
      ]
    }

In [37]:
test_dataset.to_pandas_dataframe().head(1)

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,0.37,-0.11,-0.71,-0.06,-0.65,-0.10,0.04,-0.35,-0.80,0.59,...,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00


In [33]:
data = {"data":
        [
          {"0": 0.3724756684108134, "1": -0.11356630391730027, "2": -0.7060478345151526, "3": -0.06040800875041221, "4": -0.6540846522394981, "5": -0.10477002014272219,
          "6": 0.035279110958348174, "7": -0.34837583243057135, "8": -0.8018028001600404, "9": 0.5860246521733481, "10": -1.0728719873198258, "11": 1.62293272927206,
          "12": -0.21551091417475915, "13": -0.776769403984661, "14": -1.1813162093917609, "15": -0.9439279700023309, "16": -0.9705838675587369, "17": 1.8109138810402308,
          "18": -0.11485318283789403, "19": -1.044499739955759, "20": -0.8041782736240765, "21": -0.36546741585759634, "22": 0.0976573867257759, "23": -1.182014706255381,
          "24": -0.6849259227844027, "25": -0.10346810050552385, "26": 0.36530285066800683, "27": -0.8683613109578018, "28": -0.23877727469036084, "29": -0.39582858239823265,
          "30": -0.8325132083251996, "31": -1.0381841999958483, "32": 1.163263052393548, "33": 0.20313189092138595, "34": -0.5712774912947869, "35": 0.0, "36": 0.0, "37": 0.0,
          "38": 0.0, "39": 0.0, "40": 0.0, "41": 0.0, "42": 0.0, "43": 0.0, "44": 0.0, "45": 0.0, "46": 0.0, "47": 1.0, "48": 0.0, "49": 0.0, "50": 0.0, "51": 0.0, "52": 0.0,
          "53": 0.0, "54": 0.0, "55": 0.0, "56": 0.0, "57": 0.0, "58": 0.0, "59": 0.0, "60": 0.0, "61": 1.0, "62": 0.0, "63": 0.0, "64": 0.0, "65": 0.0, "66": 0.0, "67": 0.0,
          "68": 0.0, "69": 0.0, "70": 0.0, "71": 0.0, "72": 1.0, "73": 0.0, "74": 1.0, "75": 0.0, "76": 0.0, "77": 0.0, "78": 0.0, "79": 0.0, "80": 0.0, "81": 0.0, "82": 0.0,
          "83": 0.0, "84": 0.0, "85": 0.0, "86": 0.0, "87": 0.0, "88": 0.0, "89": 0.0, "90": 0.0, "91": 0.0, "92": 1.0, "93": 0.0, "94": 0.0, "95": 0.0, "96": 1.0, "97": 0.0,
          "98": 1.0, "99": 0.0, "100": 0.0, "101": 0.0, "102": 0.0, "103": 0.0, "104": 0.0, "105": 0.0, "106": 0.0, "107": 0.0, "108": 0.0, "109": 0.0, "110": 0.0, "111": 1.0,
          "112": 0.0, "113": 0.0, "114": 0.0, "115": 0.0, "116": 0.0, "117": 1.0, "118": 0.0, "119": 0.0, "120": 0.0, "121": 1.0, "122": 0.0, "123": 0.0, "124": 1.0, "125": 0.0,
          "126": 0.0, "127": 0.0, "128": 0.0, "129": 0.0, "130": 0.0, "131": 0.0, "132": 0.0, "133": 1.0, "134": 1.0, "135": 0.0, "136": 0.0, "137": 0.0, "138": 0.0, "139": 0.0,
          "140": 0.0, "141": 0.0, "142": 1.0, "143": 0.0, "144": 0.0, "145": 0.0, "146": 1.0, "147": 0.0, "148": 1.0, "149": 0.0, "150": 0.0, "151": 0.0, "152": 0.0, "153": 0.0,
          "154": 0.0, "155": 0.0, "156": 0.0, "157": 0.0, "158": 0.0, "159": 1.0, "160": 0.0, "161": 0.0, "162": 1.0, "163": 0.0, "164": 0.0, "165": 0.0, "166": 0.0, "167": 1.0, "168": 0.0,
          "169": 0.0, "170": 0.0, "171": 0.0, "172": 1.0, "173": 0.0, "174": 0.0, "175": 0.0, "176": 0.0, "177": 0.0, "178": 0.0, "179": 1.0, "180": 0.0, "181": 0.0, "182": 0.0, "183": 0.0,
          "184": 1.0, "185": 0.0, "186": 0.0, "187": 0.0, "188": 0.0, "189": 0.0, "190": 0.0, "191": 0.0, "192": 0.0, "193": 0.0, "194": 0.0, "195": 0.0, "196": 0.0, "197": 0.0, "198": 0.0,
          "199": 0.0, "200": 0.0, "201": 0.0, "202": 0.0, "203": 0.0, "204": 0.0, "205": 0.0, "206": 0.0, "207": 0.0, "208": 0.0, "209": 0.0, "210": 0.0, "211": 0.0, "212": 0.0, "213": 0.0,
          "214": 0.0, "215": 0.0, "216": 0.0, "217": 0.0, "218": 0.0, "219": 0.0, "220": 0.0, "221": 0.0, "222": 0.0, "223": 0.0, "224": 0.0, "225": 0.0, "226": 0.0, "227": 0.0, "228": 0.0,
          "229": 0.0, "230": 0.0, "231": 0.0, "232": 0.0, "233": 0.0, "234": 0.0, "235": 0.0, "236": 0.0, "237": 0.0, "238": 0.0, "239": 0.0, "240": 0.0, "241": 0.0, "242": 0.0, "243": 0.0,
          "244": 0.0, "245": 0.0, "246": 0.0, "247": 0.0, "248": 0.0, "249": 0.0, "250": 0.0, "251": 0.0, "252": 0.0, "253": 0.0, "254": 0.0, "255": 0.0, "256": 0.0, "257": 0.0, "258": 0.0,
          "259": 0.0, "260": 0.0, "261": 0.0, "262": 0.0, "263": 0.0, "264": 0.0, "265": 0.0, "266": 0.0, "267": 0.0, "268": 0.0, "269": 0.0, "270": 0.0, "271": 1.0, "272": 0.0, "273": 0.0, 
          "274": 0.0, "275": 0.0, "276": 0.0, "277": 0.0, "278": 0.0, "279": 0.0, "280": 0.0, "281": 0.0, "282": 0.0, "283": 0.0, "284": 0.0, "285": 0.0, "286": 0.0, "287": 0.0, "288": 0.0,
          "289": 0.0, "290": 0.0, "291": 0.0, "292": 0.0, "293": 1.0, "294": 0.0, "295": 0.0, "296": 0.0, "297": 0.0, "298": 0.0, "299": 0.0, "300": 0.0, "301": 0.0, "302": 0.0, "303": 0.0, 
          "304": 0.0, "305": 0.0, "306": 0.0, "307": 0.0, "308": 0.0, "309": 1.0, "310": 0.0, "311": 0.0, "312": 0.0, "313": 0.0, "314": 1.0, "315": 0.0, "316": 0.0, "317": 0.0, "318": 0.0, 
          "319": 1.0, "320": 0.0, "321": 0.0, "322": 1.0, "323": 0.0, "324": 0.0, "325": 0.0, "326": 0.0, "327": 1.0, "328": 0.0, "329": 0.0, "330": 0.0, "331": 0.0, "332": 0.0, "333": 1.0, 
          "334": 0.0, "335": 0.0, "336": 0.0, "337": 0.0, "338": 1.0, "339": 0.0, "340": 0.0, "341": 0.0, "342": 0.0, "343": 1.0, "344": 0.0, "345": 0.0, "346": 0.0, "347": 0.0, "348": 0.0, 
          "349": 1.0, "350": 0.0, "351": 0.0, "352": 0.0, "353": 0.0, "354": 1.0, "355": 0.0, "356": 0.0, "357": 0.0, "358": 0.0, "359": 0.0, "360": 0.0, "361": 1.0, "362": 0.0, "363": 0.0, 
          "364": 0.0, "365": 1.0, "366": 0.0, "367": 0.0, "368": 0.0, "369": 0.0, "370": 0.0, "371": 0.0, "372": 0.0, "373": 0.0, "374": 1.0, "375": 0.0, "376": 0.0, "377": 0.0, "378": 0.0, 
          "379": 0.0, "380": 1.0, "381": 0.0, "382": 0.0, "383": 0.0, "384": 0.0, "385": 0.0, "386": 1.0, "387": 0.0, "388": 0.0, "389": 0.0, "390": 1.0, "391": 0.0, "392": 0.0, "393": 1.0, 
          "394": 0.0, "395": 0.0, "396": 0.0, "397": 1.0, "398": 0.0, "399": 0.0, "400": 0.0},
      ]
    }

In [43]:
my_test_values = test_dataset.to_pandas_dataframe()
test_list = my_test_values.values.tolist()

In [47]:
my_list=test_list[0]
#print(my_list)
my_data={}
print(len(my_list))
for count in range(len(my_list)):
    my_data[str(count)]=my_list[count]
print(my_data)

401
{'0': 0.3724756684108134, '1': -0.11356630391730027, '2': -0.7060478345151526, '3': -0.06040800875041221, '4': -0.6540846522394981, '5': -0.10477002014272219, '6': 0.035279110958348174, '7': -0.34837583243057135, '8': -0.8018028001600404, '9': 0.5860246521733481, '10': -1.0728719873198258, '11': 1.62293272927206, '12': -0.21551091417475915, '13': -0.776769403984661, '14': -1.1813162093917609, '15': -0.9439279700023309, '16': -0.9705838675587369, '17': 1.8109138810402308, '18': -0.11485318283789403, '19': -1.044499739955759, '20': -0.8041782736240765, '21': -0.36546741585759634, '22': 0.0976573867257759, '23': -1.182014706255381, '24': -0.6849259227844027, '25': -0.10346810050552385, '26': 0.36530285066800683, '27': -0.8683613109578018, '28': -0.23877727469036084, '29': -0.39582858239823265, '30': -0.8325132083251996, '31': -1.0381841999958483, '32': 1.163263052393548, '33': 0.20313189092138595, '34': -0.5712774912947869, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 

In [48]:
data = {"data":
        [
          my_data,
      ]
    }

In [49]:
import requests

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(service.scoring_uri, json.dumps(data), headers=headers)

print(resp)

<Response [502]>


In [50]:
import json


input_payload = json.dumps(data)

output = service.run(input_payload)

print(output)

ERROR:azureml.core.webservice.aci:Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '57', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Sun, 07 Feb 2021 18:18:38 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': 'df70a08f-5a16-4574-816c-65c205bd49bf', 'X-Ms-Run-Function-Failed': 'True'}
Content: b"float() argument must be a string or a number, not 'dict'"



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '57', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Sun, 07 Feb 2021 18:18:38 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': 'df70a08f-5a16-4574-816c-65c205bd49bf', 'X-Ms-Run-Function-Failed': 'True'}
Content: b"float() argument must be a string or a number, not 'dict'"
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '57', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Sun, 07 Feb 2021 18:18:38 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': 'df70a08f-5a16-4574-816c-65c205bd49bf', 'X-Ms-Run-Fu

TODO: In the cell below, print the logs of the web service and delete the service

In [32]:
print(service.get_logs())

2021-02-07T16:36:53,020154400+00:00 - gunicorn/run 
2021-02-07T16:36:53,021655600+00:00 - iot-server/run 
2021-02-07T16:36:53,043639300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-07T16:36:53,041043500+00:00 - rsyslog/run 
rsyslogd

In [47]:
# Delete the service
service.delete()

In [42]:
# Delete compute cluster
cpu_cluster.delete()

# References
- Cock, Dean. (2011). Ames, Iowa: Alternative to the Boston Housing Data as an End of Semester Regression Project. Journal of Statistics Education. 19. 10.1080/10691898.2011.11889627.
- [Deployment to Cloud Example](https://github.com/ErkanHatipoglu/MachineLearningNotebooks/tree/master/how-to-use-azureml/deployment/deploy-to-cloud)